# <font color=brown>**Azure Cognitive Vision REST Functions**</font>

## <font color = brown>Index</font>
***
- [Preliminaries and Libraries](#Preliminaries-and-Libraries)
- [Define Python Class for REST Processing](#Define-Python-Class-for-REST-Processing)
    - [Class initialization function](#Class-initialization-function)
    - [Preliminary data transformation functions](#Preliminary-data-transformation-functions-(needed-for-other-functions))
        - [check_degrees](#check-degrees)
        - [check_cardinality](#check-cardinality)
        - [get_dir](#get-dir)
        - [convert_stateplane](#convert-stateplane)
        - [time_convert](#time-convert)
        - [check_blob_container](#check-blob-container)
    - [Photosphere Image Operation Functions](#Photosphere-Image-Operation-Functions)
        - [get_blob_list](#get-blob-list)
        - [get_object_bounds](#get-object-bounds)
        - [draw_boxes](#draw-boxes)
        - [write_jsonfile](#write-jsonfile)
    - [Feature Functions for Image Processing and Classification](#Feature-Functions-for-Performing-Image-Processing-and-Classification-Analysis)
        - [update_blob_metadata](#update-blob-metadata)
        - [tag_photosphere_images](#tag-photosphere-images)
        - [process_cardinal_images](#process-cardinal-images)
        - [create_geojson_from_cardinals](#create-geojson-from-cardinals)

## <font color=brown>Preliminaries and Libraries</font>
***

Importing the required libraries into the project

In [ ]:
import os, io, requests, json, geojson, cv2, glob, xlrd, math, http.client, pyproj
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from decimal import Decimal
import xlsxwriter as xlw
from pandas.io.json import json_normalize
from PIL import Image, ImageDraw, ImageFont
from GPSPhoto import gpsphoto
from datetime import datetime, timedelta
import time
from tqdm import tqdm
from IPython.display import display
from azure.storage.blob import BlockBlobService

Set maximum number of http requests

In [1]:
http.client._MAXHEADERS = 5000

NameError: name 'http' is not defined

## <font color=brown>**Define Python Class for REST Processing**</font>
***

In [ ]:
class AzCognVisionRest(object):
        """Class AzCognVisionRest:
    This class contains a number of functions, methods and processes for ML object classification analysis
    using the Azure Cognitive Services Computer Vision API.

    Global Attributes
        blobAccount: the name of the Azure blob storage account
        blobKey: the API key of the Azure blob storage account
        apiRegion: the azure region of the Azure Cognitive Services Computer Vision API (e.g., 'westus')
        apiKey: the Azure Cognitive Services Computer Vision API key (from azure)
        containerName: the base container name of the Azure blob storage account containing the photosphere images

    Example Class initialization:
        az = AzCognVisionRest(blobAccount, blobKey, apiRegion, apiKey, containerName)
    """

### <font color=green>**Class initialization function**</font>

Initialization function for *AzCognVisionRest* function

In [ ]:
    def __init__(self, blobAccount, blobKey, apiRegion, apiKey, containerName):
        """Function Class Initialization
        Returns an Azure Cognitive Services Computer Vision object (REST API) using a region and key.

        Attributes
            blobAccount: the name of the Azure blob storage account
            blobKey: the API key of the Azure blob storage account
            apiRegion: the azure region of the Azure Cognitive Services Computer Vision API (e.g., 'westus')
            apiKey: the Azure Cognitive Services Computer Vision API key (from azure)
            containerName: the base container name of the Azure blob storage account containing the photosphere images

        Returns
            client: A ComputerVisionAPI object

        Notes
            This function runs on instantiation of class
        """
        # Setup account and key for the Azure blob storage containing the photosphere images
        self.blobAccount = blobAccount
        self.blobKey = blobKey
        self.blobService = BlockBlobService(self.blobAccount, self.blobKey)

        # Setup region and key for the Azure vision client API
        self.apiRegion = apiRegion
        self.subscriptionKey = apiKey
        assert self.subscriptionKey

        # Setup base url for Azure Cognitive Services Computer Vision
        self.visionBaseUrl = 'https://{}.api.cognitive.microsoft.com/vision/v2.0/'.format(self.apiRegion)

        # Setup the global headers configuration
        self.headers = {"Ocp-Apim-Subscription-Key": self.subscriptionKey, "Content-Type": "application/octet-stream"}

        # Setup the Azure blob container name
        self.containerName = containerName
        self.taggedName = '{}-tagged'.format(containerName)
        self.blobBaseUrl = 'https://{}.blob.core.windows.net'.format(self.blobAccount)
        self.blobBaseUrl_photospheres = '{}/{}'.format(self.blobBaseUrl, self.containerName)
        self.blobBaseUrl_tagged = '{}/{}'.format(self.blobBaseUrl, self.taggedName)

### <font color=green>**Preliminary data transformation functions (needed for other functions)**</font>

<div id="check-degrees" class = "alert alert-block alert-info">
    <font size = 3><b>Function: <i>check_degrees(x, y)</i></b></font>
    <p> <b>Description: </b>Checks and obtains degrees based on addition. This function cycles degrees from 0&deg; to 360&deg; based on mathematical addition. Given an initial starting degree (x), we calculate the sum between x and y. If x + y exceeds 360&deg;, the function resets the value to accomodate radial consistency.</p><br/>
    <b>Arguments:</b>
    <ul>
        <li><b>x: </b>initial(starting) degrees.</li>
        <li><b>y: </b>degrees to be added.</li>
    </ul><br/>
    <b>Outputs:</b>
    <ul><li><b>sumdeg</b>: returns the sum of degrees between 0&deg; and 360&deg;</li></ul>
</div>

In [ ]:
    def check_degrees(self, x, y):
        """Checks and obtains degrees based on addition
        This function cycles degrees from 0 to 360 based on mathematical addition.
        Given an initial starting degree (x) we calculate the sum between x and y.
        If x+y exceeds 360, the function resets the value to accomodate radial consistency.

        Arguments
            x: initial (starting) degrees
            y: degrees to be added.

        Output
            sumdeg: returns the sum of degrees between 0 and 360.
        """
        sumdeg = x + y
        if sumdeg > 360:
            sumdeg = sumdeg - 360
        return sumdeg

<div id="check-cardinality" class = "alert alert-block alert-info">
    <font size = 3><b>Function: <i>check_cardinality(value)</i></b></font>
    <p><b>Description: </b> Returns a cardinal direction from a dictionary. This function checks a direction value (in degrees, &deg;) against a cardinal direction dictionary. It returns a cardinal direction class in which the direction value belongs to.</p><br/>
    <b>Arguments:</b>
    <ul><li><b>value:</b> the direction value in degrees.</li></ul><br/>
    <b>Output:</b>
    <ul><li><b>direction:</b> the cardinal direction class label.</li></ul>
</div>

In [ ]:
    def check_cardinality(self, value):
        """Returns a cardinal direction from a dictionary
        This function checks a direction value (in degrees) against a cardinal direction dictionary
        It returns a cardinal direction class in which the direction value belongs to.

        Arguments
            value: the direction value in degrees.

        Output
            direction: the cardinal direction class label.
        """
        # Defining a cardinal directions dictionary to be used in the next function
        cardinalDictionary = {
            'N0': [0, 5.625], 'NbE': [5.625, 16.875], 'NNE': [16.875, 28.125],
            'NEbN': [28.125, 39.375], 'NE': [39.375, 50.625], 'NEbE': [50.625, 61.875],
            'ENE': [61.875, 73.125], 'EbN': [73.125, 84.375], 'E': [84.375, 95.625],
            'EbS': [95.625, 106.875], 'ESE': [106.875, 118.125], 'SEbE': [118.125, 129.375],
            'SE': [129.375, 140.625], 'SEbS': [140.625, 151.875], 'SSE': [151.875, 163.125],
            'SbE': [163.125, 174.375], 'S': [174.375, 185.625], 'SbW': [185.625, 196.875],
            'SSW': [196.875, 208.125], 'SWbS': [208.125, 219.375], 'SW': [219.375, 230.625],
            'SWbW': [230.625, 241.875], 'WSW': [241.875, 253.125], 'WbS': [253.125, 264.375],
            'W': [264.375, 275.625], 'WbN': [275.625, 286.875], 'WNW': [286.875, 298.125],
            'NWbW': [298.125, 309.375], 'NW': [309.375, 320.625], 'NWbN': [320.625, 331.875],
            'NNW': [331.875, 343.125], 'NbW': [343.125, 354.375], 'N1': [354.375, 360.000001]
            }

        for direction in cardinalDictionary:
            if cardinalDictionary[direction][0] <= round(value, 3) < cardinalDictionary[direction][1]:
                if direction == 'N0' or direction == 'N1':
                    cardinalDir = 'N'
                else:
                    cardinalDir = direction
        return cardinalDir

<div id = "get-dir" class = "alert alert-block alert-info">
    <font size=3><b>Function: <i>get_dir(easting, northing)</i></b></font>
    <p><b>Description:</b> Calculates direction from State Plane coordinates. This function calculates the direction (angle in degrees &deg;) from Easting and Northing coordinates expressed in State Plane, California zone 6 (NAD84) coordinate system.</p><br/>
    <b>Arguments:</b>
    <ul><li><b>easting:</b> Easting coordinate value in NAD84.</li>
        <li><b>northing:</b> Northing coordinate value in NAD84. </li></ul><br/>
    <b>Output:</b>
    <ul><li><b>degout:</b> direction in degrees (always positive, reverses if negative).</li></ul>
</div>

In [ ]:
    def get_dir(self, easting, northing):
        """Calculates direction from State Plane coodinates
        This function calculates the direction (angle in degrees) from Easting and Northing
        coordinates expressed in State Plane, California zone 6 (NAD84)

        Arguments
            easting: Easting coordinate value in NAD84.
            northing: Northing coordinate value in NAD84.

        Output
            degout: direction in degrees (always positive, reverses if negative)
        """
        degout = math.degrees(math.atan2(easting, northing))
        if degout >= 0:
            degout = 180 + degout
        elif degout < 0:
            degout = - degout
        return degout

<div id="convert-stateplane" class = "alert alert-block alert-info">
    <font size=3><b>Function: <i>convert_stateplane(x<sub>in</sub>, y<sub>in</sub>, z<sub>in</sub>)</i></b></font>
    <p><b>Description:</b> Converts State Plane coordinates (NAD84) to Lat-Lon degrees (WGS84). This function converts coordinates from State Plane coordinate system, California zone 6 (NAD84, espg: 2230), to default ESRI and ArcGIS online Lat-Lon degrees (WGS84, espg: 4326).</p><br/>
    <b>Arguments:</b>
    <ul><li><b>x<sub>in</sub>:</b> Easting coordinates in NAD84</li>
        <li><b>y<sub>in</sub>:</b> Northing coordinates in NAD84</li>
        <li><b>z<sub>in</sub>:</b> Elevation coordinates in NAD84</li></ul><br/>
    <b>Output:</b>
    <ul><li><b>x<sub>out</sub>:</b> Longitude coordinates in WGS84</li>
        <li><b>y<sub>out</sub>:</b> Latitude coordinates in WGS84</li>
        <li><b>z<sub>out</sub>:</b> Elevation coordinates in WGS84</li></ul><br/>
    <b>Notes:</b> <sup>(1) </sup>Setting <i>preserve_units</i> parameter as 'True' ensures we preserve the original coordinates that are provided in feet.
</div>

In [ ]:
    def convert_stateplane(self, xin, yin, zin):
        """Converts State Plane coordinates (NAD84) to Lat Lon degrees (WGS84)
        This function converts coordinates from State Plane coordinate system, CA zone 6 (NAD84, espg: 2230)
        to default ESRI and ArcGIS online Lat-Lon degrees (WGS84, espg: 4326)

        Arguments
            xin: Easting coordinates in NAD84
            yin: Northing coordinates in NAD84
            zin: Elevation coordinates in NAD84

        Output
            xout: Longitude coordinates in WGS84
            yout: Latitude coordinates in WGS84
            zout: Elevation coordinates in WGS84
        """
        # Setting preserve_units as True makes sure we preserve the original coordinates in feet.
        inProj = pyproj.Proj(init = 'epsg:2230', preserve_units = True)
        outProj = pyproj.Proj(init = 'epsg:4326')
        xout, yout, zout = pyproj.transform(inProj, outProj, xin, yin, zin)
        return (xout, yout, zout)

<div id="time-convert" class = "alert alert-block alert-info">
    <font size=3><b>Function: <i>time_convert(imgname, timestamp)</i></b></font>
    <p><b>Description:</b> Converts image timestamp string to native datetime format. This function takes as input the string timestamps from metadata and converts them to a native datetime format. The results are used in the JSON formatting where they are converted to different strings.</p><br/>
    <b>Arguments:</b>
    <ul><li><b>imgname:</b> the name of the image to be converted.</li>
        <li><b>timestamp:</b> the string timestamp input to be converted.</li></ul><br/>
    <b>Output:</b>
    <ul><li><b>dtobject:</b> a datetime object.</li></ul>
</div>

In [ ]:
    def time_convert(self, imgname, timestamp):
        """Converts image timestamp string to native datetime format
        This function takes as input the string timestamps from metadata and converts them to
        a native datetime format. The results are used in the json formatting where they are
        converted to different strings.

        Arguments
            imgname: the name of the image to be converted
            timestamp: the string timestamp input to be converted

        Output
            dtobject: a datetime object
        """
        namesplit = imgname.split('_')[0]
        YYYY = '20' + namesplit[:2]
        MM = namesplit[2:4]
        DD = namesplit[4:]
        day = timestamp / 86400
        hours = Decimal(str(day))%1 * 86400 / 3600
        minutes = Decimal(str(hours))%1 * 3600 / 60
        seconds = Decimal(str(minutes))%1 * 60
        msecs = Decimal(str(seconds))%1 * 10
        hh = int(hours)
        mm = int(minutes)
        ss = int(seconds)
        s = int(msecs)
        dtobject = datetime(int(YYYY),int(MM),int(DD),int(hh),int(mm),int(ss),100000*int(s))
        return dtobject

<div id="check-blob-container" class = "alert alert-block alert-info">
    <font size=3><b>Function: <i>check_blob_container(containerName, create=False, publicAccess='blob')</i></b></font>
    <p><b>Description:</b> Check for the presence of a blob container in the account. This function checks the Azure storage account whether or not a blob container (folder) exists or not. If the container exists, the program makes sure the <i>publicAccess</i> is set to the value of the function. If the container does not exist, if <i>create = True</i>, then the folder is created and <i>publicAccess</i> is set. If the container does not exist, and <i>crate = False</i> (default), nothing is done.</p><br/>
    <b>Arguments:</b>
    <ul><li><b>containerName:</b> the name of the blob container (folder) to be checked.</li>
        <li><b>create (=False by default):</b> whether or not to create a new container if it does not exist.</li>
        <li><b>publicAccess (='blob' by default):</b> level of public access to URL ('blob', 'container', etc.)</li></ul><br/>
    <b>Output:</b>
    <ul><li><b>Nothing</b>. Performs operations in Microsoft Azure Storage on the cloud.</li></ul>
</div>

In [ ]:
    def check_blob_container(self, containerName, create=False, publicAccess='blob'):
        """Check for the presence of a blob container in the account
        This function checks the Azure storage account whether or not a blob container (folder) exists or not.
        If the container exists, the program makes sure the publicAccess is set to the value of the function.
        If the container does not exist, if create=True, then the folder is created and publicAccess is set.
        If the container does not exist, and create=False (default), nothing is done.

        Arguments
            containerName: the name of the blob container (folder) to be checked
            create (=False by default): whether or not to create a new container if it doesn't exist.
            publicAccess (='blob' by default): level of public access to URL ('blob', 'container', etc)

        Returns
            Nothing. Performs operations in Microsoft Azure Storage on the cloud.
        """
        if self.blobService.exists(containerName):
            self.blobService.set_container_acl(containerName, public_access = publicAccess)
            print('Container {} exists. Public Access is set to {}'.format(containerName, publicAccess))
        elif create == True:
            self.blobService.create_container(containerName, public_access = publicAccess)
            assert self.blobService.exists(containerName)
            print('Container {} did not exist. A new container is created with public_access set to {}'.format(containerName, publicAccess))
        else:
            print('Container did not exist. No changes are requested. Program exits.')
        return

### <font color=green>**Photosphere Image Operation Functions**</font>

<div id="get-blob-list" class = "alert alert-block alert-info">
    <font size=3><b>Function: <i>get_blob_list(containerName=None)</i></b></font>
    <p><b>Description:</b> Lists all blobs in Azure storage blob. This function gets a list of all files in an Azure storage blob (by container folder name).</p><br/>
    <b>Arguments:</b>
    <ul><li><b>containerName (=None):</b> If <i>containerName=None</i>, the function uses the Azure storage blob container name (from class initialization)</li>
        <li><b>containerName (=defined):</b> if <i>contanerName</i> is defined (not None), then the function uses the defined Azure storage blob container.</li></ul><br/>
    <b>Output:</b>
    <ul><li><b>blobList:</b> the list of all files in the container.</li></ul>
</div>

In [ ]:
    def get_blob_list(self, containerName=None):
        """List all blobs in Azure storage blob
        This function gets a list of all files in an Azure storage blob (by container folder name)

        Arguments
            containerName (optional): 
                if containerName is None: Uses the Azure storage blob container name (from class initialization)
                if containerName is not None: Uses the defined Azure storage blob container

        Output
            blobList: the list of all files in the container
        """
        # List the blobs in the container (from class initialization)
        if containerName is None:
            container = self.containerName
        else:
            container = containerName
        blobList = []
        generator = self.blobService.list_blobs(container)
        for blob in generator:
            blobList.append(blob)
        return blobList

<div id="get-object-bounds" class = "alert alert-block alert-info">
    <font size=3><b>Function: <i>get_object_bounds(jsonstring)</i></b></font>
    <p><b>Description:</b> Obtains detected object bounds from bounding box coordinates. This function returns bounding coordinates for an object in detected Azure cognitive vision JSON string response.</p><br/>
    <b>Arguments:</b>
    <ul><li><b>jsonstring:</b> the JSON detection response from Azure Cognitive Vision containing the object.</li></ul><br/>
    <b>Output:</b>
    <ul><li><b>bounds:</b> the set of bounds expressed in bounding box polygon coordinates (x, y, w, h).</li></ul>
</div>

In [ ]:
    def get_object_bounds(self, jsonstring):
        """Get detected object bounds from bounding box coordinates
        This function returns bounding coordinates for an object in detected Azure cognitive vision
        json string.

        Arguments
            jsonstring: the json detection response containing the object

        Output
            bounds: the set of bounds expressed in bounding box coordinates (x, y, w, h)
        """
        bounds = []
        nobj = jsonstring['Number_of_Objects']
        for i in range(0, nobj):
            bounds.append({
                'object': jsonstring['Object_{}'.format(i+1)],
                'vertices': [
                    {'x': jsonstring['x{}'.format(i+1)]},
                    {'y': jsonstring['y{}'.format(i+1)]},
                    {'w': jsonstring['w{}'.format(i+1)]},
                    {'h': jsonstring['h{}'.format(i+1)]}
                ]
            })
        return bounds

<div id="draw-boxes" class = "alert alert-block alert-info">
    <font size=3><b>Function: <i>draw_boxes(image, bounds)</i></b></font>
    <p><b>Description:</b> Draws annotation boxes in image. This function uses the bound coordinates in order to draw annotation boxes around photosphere images.</p><br/>
    <b>Arguments:</b>
    <ul><li><b>image:</b> the photosphere image to be annotated (cardinal)</li>
        <li><b>bounds:</b> the bounding box coordinates of the detected objects (obtained through the <i>get_object_bounds</i> function).</li></ul><br/>
    <b>Output:</b>
    <ul><li><b>image:</b> the annotated image.</li></ul>
</div>

In [ ]:
    def draw_boxes(self, image, bounds):
        """Draws annotation boxes in image
        This function uses the bound coordinates to draw annotation boxes around photosphere images

        Arguments
            image: the photosphere image to be annotated (cardinal)
            bounds: the bounding box coordinates of the detected objects

        Output
            image: the annotated image
        """
        draw = ImageDraw.Draw(image)
        font = ImageFont.truetype('arial.ttf', 18)
        for bound in bounds:
            draw.rectangle([
                bound['vertices'][0]['x'],
                bound['vertices'][1]['y'],
                bound['vertices'][0]['x'] + bound['vertices'][2]['w'],
                bound['vertices'][1]['y'] + bound['vertices'][3]['h']
                ], None, 'red')
            draw.text((bound['vertices'][0]['x'] + 5, bound['vertices'][1]['y'] + 5),
                bound['object'], fill = 'red', font = font)
        return image

<div id="write-jsonfile" class = "alert alert-block alert-info">
    <font size=3><b>Function: <i>write_jsonfile(name, data)</i></b></font>
    <p><b>Description:</b> Writes detection output into a JSON file. This function outputs the processed results of the Azure cognitive vision detection process into a JSON file.</p><br/>
    <b>Arguments:</b>
    <ul><li><b>name:</b> the name of the JSON file to be saved.</li>
        <li><b>data:</b> the JSON string response data to be included.</li></ul><br/>
    <b>Output:</b>
    <ul><li><b>Nothing.</b> the JSON file is saved using the name provided</li></ul>
</div>

In [ ]:
    def write_jsonfile(self, name, data):
        """Writes detection output into a jsonfile
        This function outputs the processed results of the Azure cognitive vision detection process
        into a json file.

        Arguments
            name: the name of the json file to be saved
            data: the json string response data to be included

        Output
            Nothing, the json file is saved using the name provided
        """
        filename = name + '.json'
        with open(filename, 'w') as fp:
            json.dump(data, fp)
        return

### <font color=green>**Feature Functions for Performing Image Processing and Classification Analysis**</font>

<div id="update-blob-metadata" class = "alert alert-block alert-info">
    <font size=3><b>Function: <i>update_blob_metadata(metadata='CameraMetadata.xlsx')</i></b></font>
    <p><b>Description:</b> Uploads and updates blob metadata from excel file metadata. This function will upload and update the blob metadata, based on the metadata file stored for the photosphere image collection.</p><br/>
    <b>Arguments:</b>
    <ul><li><b>metadatafile:</b> the metadata file name (default = 'CameraMetadata.xlsx')</li></ul><br/>
    <b>Output:</b>
    <ul><li><b>Nothing.</b> Performs operation in the blob container</li></ul>
</div>

In [ ]:
    def update_blob_metadata(self, metadata = 'CameraMetadata.xlsx'):
        """Uploads and updates blob metadata from excel file metadata
        This function will upload and update the blob metadata, based on
        the metadata file stored in image.

        Arguments
            metadatafile: the metadata filename

        Output
            Nothing; performs operation in the blob container
        """
        containerList = self.get_blob_list()
        self.check_blob_container(self.containerName)
        noImg = len(containerList)
        print('Number of blobs in container: {}'.format(noImg))

        if metadata is not None:
            xlMetadata = pd.read_excel(metadata, sheet_name=0)
            for i, img in enumerate(containerList):
                imgName = img.name
                print('\tProcessing image ({} of {}): {}'.format(i+1, noImg, imgName))
                xlMetaImg = xlMetadata.loc[xlMetadata['Filename'] == imgName]
                xlcols = xlMetaImg.iloc[0]

                # Convert the State Plane to Lat-Lon coordinates
                lon, lat, alt = self.convert_stateplane(
                    xlcols['OriginEasting'],
                    xlcols['OriginNorthing'],
                    xlcols['OriginHeight']
                    )

                # Create an empty json string to hold the image metadata
                jsonimg = {}
                jsonimg['Photosphere_Image_Name'] = xlcols['Filename']
                imgdt = self.time_convert(xlcols['Filename'], xlcols['Timestamp'])
                jsonimg['DateTime_display'] = imgdt.strftime('%m/%d/%Y %H:%M:%S.%f').rstrip('0')
                jsonimg['DateTime_string'] = imgdt.strftime('%Y%m%d%H%M%S.%f').rstrip('0')
                jsonimg['Photosphere_Resolution'] = '8000 x 4000'
                jsonimg['Photosphere_URL'] = '{}/{}'.format(self.blobBaseUrl_photospheres, xlcols['Filename'])
                jsonimg['Photosphere_Tagged_URL'] = '{}/{}'.format(self.blobBaseUrl_tagged, xlcols['Filename'])
                jsonimg['Longitude'] = lon
                jsonimg['Latitude'] = lat
                jsonimg['Altitude'] = alt
                jsonimg['Direction'] = self.get_dir(xlcols['DirectionEasting'],xlcols['DirectionNorthing'])
                jsonimg['Origin_Easting'] = xlcols['OriginEasting']
                jsonimg['Origin_Northing'] = xlcols['OriginNorthing']
                jsonimg['Origin_Height'] = xlcols['OriginHeight']
                jsonimg['Direction_Easting'] = xlcols['DirectionEasting']
                jsonimg['Direction_Northing'] = xlcols['DirectionNorthing']
                jsonimg['Direction_Height'] = xlcols['DirectionHeight']
                jsonimg['Up_Easting'] = xlcols['UpEasting']
                jsonimg['Up_Northing'] = xlcols['UpNorthing']
                jsonimg['Up_Height'] = xlcols['UpHeight']
                jsonimg['Roll'] = xlcols['Roll']
                jsonimg['Pitch'] = xlcols['Pitch']
                jsonimg['Yaw'] = xlcols['Yaw']
                jsonimg['Omega'] = xlcols['Omega']
                jsonimg['Phi'] = xlcols['Phi']
                jsonimg['Kappa'] = xlcols['Kappa']
                metastring = {}
                for key in jsonimg.keys():
                    metastring[key] = str(jsonimg[key])
                self.blobService.set_blob_metadata(self.containerName,imgName, metastring)
                print('\tUpdating metadata in azure blob')
        return

<div id="tag-photosphere-images" class = "alert alert-block alert-info">
    <font size=3><b>Function: <i>tag_photosphere_images(blobContainerOut)</i></b></font>
    <p><b>Description:</b> Creates tagged photosphere images. This function tags and annotates the original photosphere images with the areas coverted by the cardinal areas. The tagged photosphere images are then copied to a new blob container.</p><br/>
    <b>Arguments</b>
    <ul><li><b>blobContainerOut:</b> the blob container for the tagged images to be saved (if it does not exist, then the folder is created.</li></ul><br/>
    <b>Output</b>
    <ul><li><b>Nothing.</b> All operations are done to the blob container.</li></ul>
</div>

In [ ]:
    def tag_photosphere_images(self, blobContainerOut):
        """Creates tagged photosphere images
        This function tags and annotates the original photosphere images with the areas covered by
        the cardinal areas. The tagged photosphere images are then copied to a new blob container.

        Arguments
            blobContainerOut: the blob container for the tagged images to be saved (if it doesn't
                            exist, then the folder is created)

        Output
            Nothing, all operations are done to the blob container.
        """
        try:
            self.check_blob_container(self.containerName)
            self.check_blob_container(blobContainerOut, create=True, publicAccess='blob')
            blobListIn = self.get_blob_list(self.containerName)
            for blob in tqdm(blobListIn):
                blobmeta = self.blobService.get_blob_metadata(self.containerName,blob.name)
                imageName = blob.name
                content = self.blobService.get_blob_to_bytes(self.containerName, imageName).content
                img = Image.open(io.BytesIO(content))
                draw = ImageDraw.Draw(img)
                font = ImageFont.truetype('arial.ttf', 45)
                areas = []
                step = 1000
                for i in range(0, 8000, step):
                    coor = (i, 1550, i + step, 2550)
                    areas.append(coor)
                for ncard, area in enumerate(areas):
                    draw.rectangle([area[0], area[1], area[2], area[3]], None, 'red', width=3)
                    draw.text((area[0] + 5, area[1] + 5), str(ncard + 1), fill = 'red', font = font)
                    taggedImgArray = io.BytesIO()
                    img.save(taggedImgArray, format = 'JPEG')
                    taggedImgArray = taggedImgArray.getvalue()
                    self.blobService.create_blob_from_bytes(
                        container_name = blobContainerOut,
                        blob_name = imageName,
                        blob = taggedImgArray,
                        metadata = blobmeta
                        )
        except Exception as ex:
            print(ex.args[0])

<div id="process-cardinal-images" class = "alert alert-block alert-info">
    <font size=3><b>Function: <i>process_cardinal_images(blob, containerIn, containerTagged, containerOut)</i></b></font>
    <p><b>Description:</b> Processes cardinal images from original photosphere images. This function crops and obtains 8 cardinal images (1000 x 1000) from the original photospheres, by cropping a region between 1550 and 2550 pixels, i.e., from (x<sub>1</sub> = 0, y<sub>1</sub> = 1550), to (x<sub>2</sub> = 8000, y<sub>2</sub>) vertically. The function returns a list with 8 cardinal images, from left to right, each image covering a 45&deg; vision span.</p><br/>
    <b>Arguments</b>
    <ul><li><b>blob:</b> the blob object (photosphere image) to be processed</li>
        <li><b>containerIn:</b> the name of the blob storage container containing the blob.</li>
        <li><b>containerTagged:</b> the name of the blob storage container containing the tagged photosphere images.</li>
        <li><b>containerOut:</b> the name of the blob storage container for the cardinal images to be saved.</li></ul><br/>
    <b>Output:</b>
    <ul><li><b>Nothing.</b> The results are processed and saved in the <i>containerOut</i> storage blob.</li></ul>
</div>

In [ ]:
    def process_cardinal_images(self, blob, containerIn, containerTagged, containerOut):
        """Processes cardinal images from original photospheres
        This function crops and obtains 8 cardinal images (1000 x 1000) from the original photospheres,
        by cropping a region between 1550 and 2550 pixels, i.e., from (x1 = 0, y1 = 1550) to (x2 = 8000, 
        y2 = 2550) vertically. The function returns a list with 8 cardinal images, from left to right, 
        each image covering a 45 degrees vision span.

        Arguments
            blob: the blob object (photosphere image) to be processed
            containerIn: the name of the blob storage container containing the blob
            containerOut: the name of the blob storage container for the cardinal images to be saved

        Output
            Nothing; the results are processed and saved in containerOut storage blob.
        """
        try:
            imageName = blob.name
            
            # Getting the photosphere image metadata
            metaString = {}
            if self.blobService.get_blob_metadata(containerIn, imageName) is not {}:
                metaString = self.blobService.get_blob_metadata(containerIn, imageName)
                fields = ['Direction', 'Longitude', 'Latitude', 'Altitude', 'Origin_Easting', 'Origin_Northing', 'Origin_Height', 'Direction_Easting', 'Direction_Northing', 'Direction_Height', 'Up_Easting', 'Up_Northing', 'Up_Height', 'Roll', 'Pitch', 'Yaw', 'Omega', 'Phi', 'Kappa']
                for field in fields:
                    metaString[field] = float(metaString[field])

                # Getting the photosphere image from azure blob storage and convert it to bytes
                content = self.blobService.get_blob_to_bytes(containerIn, imageName).content
                img = Image.open(io.BytesIO(content))

                # Creating the areas of the cardinal images
                areas = []
                step = 1000
                for i in range(0, 8000, step):
                    coor = (i, 1550, i+step, 2550)
                    areas.append(coor)

                # This is the loop for the 8 cardinal areas
                for ncard, area in enumerate(areas):
                    cmeta = {}
                    cmeta = metaString
                    cardinalImg = img.crop(area)
                    cardinalArray = io.BytesIO()
                    cardinalImg.save(cardinalArray, format='JPEG')
                    cardinalArray = cardinalArray.getvalue()
                    cardinalDir = self.check_degrees(cmeta['Direction'], 22.5)
                    cardinalDir = self.check_degrees(cardinalDir, ncard * 45.0)
                    cardinalLabel = self.check_cardinality(cardinalDir)
                    cardinalImgName = '{}_{}_{}.jpg'.format(imageName.split('.jpg')[0], ncard + 1, cardinalLabel)
                    cmeta['Cardinal_Image_Name'] = cardinalImgName
                    cmeta['Cardinal_Image_URL'] = '{}/{}/{}'.format(self.blobBaseUrl, containerOut, cardinalImgName)
                    cmeta['Cardinal_Number'] = ncard + 1
                    cmeta['Cardinal_Direction'] = cardinalDir
                    cmeta['Cardinal_Direction_Label'] = cardinalLabel

                    # Set up the Computer Vision analysis parameter
                    url = self.visionBaseUrl + 'analyze'
                    headers = self.headers
                    params = {"visualFeatures": "Categories,Tags,Description,ImageType,Color,Objects"}
                    response = requests.post(url, headers = headers, params = params, data = cardinalArray)
                    response.raise_for_status()
                    responsejson = response.json()
                    if 'captions' in responsejson['description']:
                        if responsejson['description']['captions']:
                            cmeta['Caption'] = responsejson['description']['captions'][0]['text']
                            cmeta['Caption_Confidence'] = responsejson['description']['captions'][0]['confidence']
                    if 'metadata' in responsejson:
                        cmeta['Image_Width'] = responsejson['metadata']['width']
                        cmeta['Image_Height'] = responsejson['metadata']['height']
                        cmeta['Image_Format'] = responsejson['metadata']['format']
                    if 'imageType' in responsejson:
                        cmeta['Clip_Art_Type'] = responsejson['imageType']['clipArtType']
                        cmeta['Line_Drawing_Type'] = responsejson['imageType']['lineDrawingType']
                    if 'color' in responsejson:
                        cmeta['Dominant_Color_Foreground'] = responsejson['color']['dominantColorForeground']
                        cmeta['Dominant_Color_Background'] = responsejson['color']['dominantColorBackground']
                        if len(responsejson['color']['dominantColors']) > 1:
                            cmeta['Dominant_Colors'] = ','.join(responsejson['color']['dominantColors'])
                        elif len(responsejson['color']['dominantColors']) == 1:
                            cmeta['Dominant_Colors'] = responsejson['color']['dominantColors'][0]
                        elif responsejson['color']['dominantColors'] is None:
                            cmeta['Dominant_Colors'] = ''
                    if 'categories' in responsejson:
                        lcat = len(responsejson['categories'])
                        cmeta['Number_of_Categories'] = lcat
                        for ncat, obj in enumerate(responsejson['categories']):
                            for cat in obj:
                                catName = 'Category_{}_{}'.format(cat.capitalize(), str(ncat + 1))
                                cmeta[catName] = obj[cat]
                    if 'tags' in responsejson:
                        ltags = len(responsejson['tags'])
                        cmeta['Number_of_Tags'] = ltags
                        for ntag, obj in enumerate(responsejson['tags']):
                            for tag in obj:
                                tagName = 'Tag_{}_{}'.format(tag.capitalize(), str(ntag + 1))
                                cmeta[tagName] = obj[tag]
                    if 'tags' in responsejson['description']:
                        cmeta['Description_Tags'] = ','.join(responsejson['description']['tags'])
                    if 'objects' in responsejson:
                        lobj = len(responsejson['objects'])
                        cmeta['Number_of_Objects'] = lobj
                        for nobj, obj in enumerate(responsejson['objects']):
                            centerX = obj['rectangle']['x'] + (obj['rectangle']['w'] / 2)
                            centerY = obj['rectangle']['y'] + (obj['rectangle']['h'] / 2)
                            centerDir = cardinalDir = 22.5 + (centerX * 0.045)
                            cmeta['Object_{}'.format(nobj + 1)] = obj['object']
                            cmeta['Object_{}_Confidence'.format(nobj + 1)] = obj['confidence']
                            cmeta['Object_{}_Direction'.format(nobj + 1)] = centerDir
                            cmeta['Object_{}_Longitude'.format(nobj + 1)] = 0.00
                            cmeta['Object_{}_Latitude'.format(nobj + 1)] = 0.00
                            cmeta['x{}'.format(nobj + 1)] = obj['rectangle']['x']
                            cmeta['y{}'.format(nobj + 1)] = obj['rectangle']['y']
                            cmeta['w{}'.format(nobj + 1)] = obj['rectangle']['w']
                            cmeta['h{}'.format(nobj + 1)] = obj['rectangle']['h']
                            cmeta['Center_x{}'.format(nobj + 1)] = centerX
                            cmeta['Center_y{}'.format(nobj + 1)] = centerY
                            if 'parent' in obj:
                                lpar = len(obj['parent'])
                                if lpar == 0:
                                    nparents = 0
                                else:
                                    nparents = lpar - 1
                                    k = 1
                                    cmeta['Object_{}_Parent_{}'.format(nobj + 1, k)] = obj['parent']['object']
                                    cmeta['Object_{}_Parent_{}_Confidence'.format(nobj + 1, k)] = obj['parent']['confidence']
                                    if 'parent' in obj['parent']:
                                        k += 1
                                        cmeta['Object_{}_Parent_{}'.format(nobj + 1, k)] = obj['parent']['parent']['object']
                                        cmeta['Object_{}_Parent_{}_Confidence'.format(nobj + 1, k)] = obj['parent']['parent']['confidence']
                                        if 'parent' in obj['parent']['parent']:
                                            k += 1
                                            cmeta['Object_{}_Parent_{}'.format(nobj + 1, k)] = obj['parent']['parent']['parent']['object']
                                            cmeta['Object_{}_Parent_{}_Confidence'.format(nobj + 1, k)] = obj['parent']['parent']['parent']['confidence']
                                            if 'parent' in obj['parent']['parent']['parent']:
                                                k += 1
                                                cmeta['Object_{}_Parent_{}'.format(nobj + 1, k)] = obj['parent']['parent']['parent']['parent']['object']
                                                cmeta['Object_{}_Parent_{}_Confidence'.format(nobj + 1, k)] = obj['parent']['parent']['parent']['parent']['confidence']

                    bounds = self.get_object_bounds(cmeta)
                    taggedImg = self.draw_boxes(cardinalImg, bounds)
                    taggedArray = io.BytesIO()
                    taggedImg.save(taggedArray, format='JPEG')
                    taggedArray = taggedArray.getvalue()

                    cardinalMetaBlob = {}
                    for key in cmeta.keys():
                        cardinalMetaBlob[key] = str(cmeta[key])

                    self.blobService.create_blob_from_bytes(
                        container_name = containerOut,
                        blob_name = cardinalImgName,
                        blob = taggedArray,
                        metadata = cardinalMetaBlob
                        )
            return
        except Exception as ex:
            # Print the exception message
            print(ex.args[0])

<div id="create-geojson-from-cardinals" class = "alert alert-block alert-info">
    <font size=3><b>Function: <i>create_geojson_from_cardinals(container)</i></b></font>
    <p><b>Description:</b> Generates a GeoJSON string from cardinal photosphere image analysis. This function follows the <i>process_cardinal_images</i> function after the cardinal images are generated, their object detection process from Azure Cognitive Services Computer Vision is completed, and the cardinal images have been annotated and tagged.</p><br/>
    <b>Arguments:</b>
    <ul><li><b>container:</b> The Azure blob storage container that holds the cardinal images (analyzed).</li></ul><br/>
    <b>Output:</b>
    <ul><li><b>fcresponse:</b> a GeoJSON Feature Collection containing all GeoJSON features and geopoints with all analyses.</li></ul>
</div>

In [ ]:
    def create_geojson_from_cardinals(self, container):
        """Generates a GeoJSON String from cardinal photosphere image analysis
        This function follows the process_cardinal_images function after the cardinal images are generated,
        their object detection process from Azure cognitive services computer vision is completed, and the 
        cardinal images have been annotated and tagged.

        Arguments
            container: the Azure blob storage container that holds the cardinal images (analyzed)

        Returns
            fcresponse: a GeoJSON Feature Collection containing all GeoJSON features and geopoints with all analyses.
        """
        try:
            featList = []
            self.check_blob_container(container)
            blobList = self.get_blob_list(container)
            
            for blob in tqdm(blobList):
                if self.blobService.get_blob_metadata(container, blob.name) is not {}:
                    metaString = self.blobService.get_blob_metadata(container, blob.name)

                    fieldsFloat = ['Direction', 'Longitude', 'Latitude', 'Altitude', 'Origin_Easting', 'Origin_Northing', 'Origin_Height', 'Direction_Easting', 'Direction_Northing', 'Direction_Height', 'Up_Easting', 'Up_Northing', 'Up_Height', 'Roll', 'Pitch', 'Yaw', 'Omega', 'Phi', 'Kappa', 'Cardinal_Direction', 'Caption_Confidence']
                    for fieldFloat in fieldsFloat:
                        if fieldFloat in metaString:
                            metaString[fieldFloat] = float(metaString[fieldFloat])

                    fieldsInt = ['Cardinal_Number', 'Image_Width', 'Image_Height', 'Number_of_Categories', 'Number_of_Tags', 'Number_of_Objects']
                    for fieldInt in fieldsInt:
                        if fieldInt in metaString:
                            metaString[fieldInt] = int(metaString[fieldInt])

                    if metaString['Number_of_Categories'] >= 1:
                        for i in range(1, metaString['Number_of_Categories'] + 1):
                            metaString['Category_Score_{}'.format(i)] = float(metaString['Category_Score_{}'.format(i)])
                    if metaString['Number_of_Tags'] >= 1:
                        for j in range(1, metaString['Number_of_Tags'] + 1):
                            metaString['Tag_Confidence_{}'.format(j)] = float(metaString['Tag_Confidence_{}'.format(j)])
                    if metaString['Number_of_Objects'] >= 1:
                        for k in range(1, metaString['Number_of_Objects'] + 1):
                            metaString['Object_{}_Confidence'.format(k)] = float(metaString['Object_{}_Confidence'.format(k)])
                            metaString['Object_{}_Direction'.format(k)] = float(metaString['Object_{}_Direction'.format(k)])
                            metaString['Object_{}_Longitude'.format(k)] = float(0.0)
                            metaString['Object_{}_Latitude'.format(k)] = float(0.0)
                            metaString['x{}'.format(k)] = int(metaString['x{}'.format(k)])
                            metaString['y{}'.format(k)] = int(metaString['y{}'.format(k)])
                            metaString['w{}'.format(k)] = int(metaString['w{}'.format(k)])
                            metaString['h{}'.format(k)] = int(metaString['h{}'.format(k)])
                            metaString['Center_x{}'.format(k)] = float(metaString['Center_x{}'.format(k)])
                            metaString['Center_y{}'.format(k)] = float(metaString['Center_y{}'.format(k)])
                    for item in metaString:
                        tempURL = metaString['Photosphere_URL'].split('/')
                        tempURL[3] = 'photospheres-tagged'
                        metaString['Photosphere_URL'] = ('/').join(tempURL)

                    gpoint = geojson.Point((metaString['Longitude'], metaString['Latitude']))
                    gfeature = geojson.Feature(geometry = gpoint, properties = metaString)
                    featList.append(gfeature)
            fcresponse = geojson.FeatureCollection(featList)
            return fcresponse
        except Exception as ex:
            # Print the exception message
            print(ex.args[0])